In [1]:
!pip install torch
!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!pip install git+https://github.com/Dao-AILab/causal-conv1d

  Cloning https://github.com/Dao-AILab/causal-conv1d to /tmp/pip-req-build-ehe65pp4
  Running command git clone --filter=blob:none --quiet https://github.com/Dao-AILab/causal-conv1d /tmp/pip-req-build-ehe65pp4
  Resolved https://github.com/Dao-AILab/causal-conv1d to commit e940ead2fd962c56854455017541384909ca669f
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 11.1 MB/s eta 0:00:00
  Created wheel for causal_conv1d: filename=causal_conv1d-1.5.0.post8-cp311-cp311-linux_x86_64.whl size=103984203 sha256=f1bff4a48017e4a1b2f18a84b3d881df7de2cff8cc9dbbc0c53c9d5df233359b
  Stored in directory: /tmp/pip-ephem-wheel-cache-t7b6gp4j/wheels/78/92/4d/c421024d7eef5e9a7a05dc488f090c3bddb325455544821101
Successfully built causal_conv1d


In [3]:
!git clone https://github.com/state-spaces/mamba.git

Cloning into 'mamba'...
remote: Enumerating objects: 739, done.
remote: Total 739 (delta 0), reused 0 (delta 0), pack-reused 739 (from 1)
Receiving objects: 100% (739/739), 1.55 MiB | 4.17 MiB/s, done.
Resolving deltas: 100% (405/405), done.


In [4]:
%cd mamba

/content/mamba


In [5]:
!export CAUSAL_CONV1D_FORCE_BUILD=TRUE
!export CAUSAL_CONV1D_SKIP_CUDA_BUILD=TRUE
!export CAUSAL_CONV1D_FORCE_CXX11_ABI=TRUE

In [6]:
!pip show causal-conv1d

Name: causal-conv1d
Version: 1.5.0.post8
Summary: Causal depthwise conv1d in CUDA, with a PyTorch interface
Home-page: https://github.com/Dao-AILab/causal-conv1d
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: ninja, packaging, torch
Required-by: 


In [7]:
!pip install . --no-build-isolation

Processing /content/mamba
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mamba_ssm: filename=mamba_ssm-2.2.4-cp311-cp311-linux_x86_64.whl size=323672993 sha256=8a0be01153fa30727a9e69024fbe061eb92c7ba4416d2049c5fc3107ed91d852
  Stored in directory: /tmp/pip-ephem-wheel-cache-49t1ohmp/wheels/d1/52/ed/0424436ff562924889b52e36e9848d82b09b3dfd1c3c675f47
Successfully built mamba_ssm


In [16]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import VGG16_Weights
from mamba_ssm import Mamba
import os
import sys


class InceptionBlock(nn.Module):
    def __init__(self, in_channels=128):
        super(InceptionBlock, self).__init__()
        self.branch1x1 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 96, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 128, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 96, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 96, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 192, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.BatchNorm2d(192),
            nn.Conv2d(192, 192, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(192)
        )
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )

    def forward(self, x):
        b1 = self.branch1x1(x)
        b2 = self.branch3x3(x)
        b3 = self.branch5x5(x)
        b4 = self.branch_pool(x)
        return torch.cat([b1, b2, b3, b4], dim=1)

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, patch_size=5, emb_size=16):
        super().__init__()
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.proj = nn.Linear(in_channels * patch_size * patch_size, emb_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.permute(0, 2, 3, 1, 4, 5).contiguous()
        x = x.view(B, -1, C * self.patch_size * self.patch_size)
        return self.proj(x)

class MambaBlock(nn.Module):
    def __init__(self, emb_size=16, d_state=64, d_conv=4, expand=4):
        super().__init__()
        self.norm = nn.LayerNorm(emb_size)
        self.mamba = Mamba(
            d_model=emb_size,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )
        print("Mamba instance:", self.mamba)
        self.mlp = nn.Sequential(
            nn.Linear(emb_size, emb_size * 2),
            nn.GELU(),
            nn.Linear(emb_size * 2, emb_size)
        )
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        residual = x
        x = self.norm(x)
        print("Input to Mamba:", x.shape)
        print("Mamba callable:", callable(self.mamba))
        print("Type of conv1d inside Mamba:", type(self.mamba.conv1d))
        x = self.mamba(x)
        x = residual + x
        x = x + self.mlp(self.norm(x))
        return self.dropout(x)

class PlantXMamba(nn.Module):
    def __init__(self, num_classes=4, patch_size=5, emb_size=16, num_blocks=4, dropout=0.1):
        super().__init__()
        vgg = models.vgg16(weights=VGG16_Weights.DEFAULT)
        self.vgg_block = nn.Sequential(*list(vgg.features[:10]))
        self.inception = InceptionBlock(in_channels=128)
        self.patch_embed = PatchEmbedding(in_channels=512, patch_size=patch_size, emb_size=emb_size)
        self.transformer = nn.Sequential(*[MambaBlock(emb_size=emb_size, d_state=64, d_conv=4, expand=4) for _ in range(num_blocks)])
        self.norm = nn.LayerNorm(emb_size)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        x = self.vgg_block(x)
        x = self.inception(x)
        x = self.patch_embed(x)
        x = self.transformer(x)
        x = self.norm(x)
        x = x.permute(0, 2, 1)
        x = self.global_pool(x).squeeze(-1)
        return self.classifier(x)

In [17]:
def test_plantxmamba(batch_size, device, input_size=(3, 224, 224), num_classes=4, patch_size=5, emb_size=16):
    print(f"Testing PlantXMamba with batch_size={batch_size} on device={device}")

    model = PlantXMamba(
        num_classes=num_classes,
        patch_size=patch_size,
        emb_size=emb_size,
        num_blocks=4,
        dropout=0.1
    ).to(device)

    model_device = next(model.parameters()).device
    print(f"Model is on device: {model_device}")

    x = torch.randn(batch_size, *input_size).to(device)
    print(f"Input shape: {x.shape}, Input device: {x.device}")

    output = None
    try:
        model.eval()
        with torch.no_grad():
            output = model(x)
        print(f"Output shape: {output.shape}")
        print(f"Output device: {output.device}")

        expected_output_shape = (batch_size, num_classes)
        assert output.shape == expected_output_shape, \
            f"Expected output shape {expected_output_shape}, but got {output.shape}"
        print("Test passed successfully!")

    except Exception as e:
        print(f"Error during testing: {str(e)}")

    return output

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    print("\n--- Test Case 1: Batch size = 1 ---")
    output1 = test_plantxmamba(batch_size=1, device=device, patch_size=5, emb_size=16)

    print("\n--- Test Case 2: Batch size = 4 ---")
    output2 = test_plantxmamba(batch_size=4, device=device, patch_size=5, emb_size=16)

Using device: cuda

--- Test Case 1: Batch size = 1 ---
Testing PlantXMamba with batch_size=1 on device=cuda


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 75.6MB/s]


Mamba instance: Mamba(
  (in_proj): Linear(in_features=16, out_features=128, bias=False)
  (conv1d): Conv1d(64, 64, kernel_size=(4,), stride=(1,), padding=(3,), groups=64)
  (act): SiLU()
  (x_proj): Linear(in_features=64, out_features=129, bias=False)
  (dt_proj): Linear(in_features=1, out_features=64, bias=True)
  (out_proj): Linear(in_features=64, out_features=16, bias=False)
)
Mamba instance: Mamba(
  (in_proj): Linear(in_features=16, out_features=128, bias=False)
  (conv1d): Conv1d(64, 64, kernel_size=(4,), stride=(1,), padding=(3,), groups=64)
  (act): SiLU()
  (x_proj): Linear(in_features=64, out_features=129, bias=False)
  (dt_proj): Linear(in_features=1, out_features=64, bias=True)
  (out_proj): Linear(in_features=64, out_features=16, bias=False)
)
Mamba instance: Mamba(
  (in_proj): Linear(in_features=16, out_features=128, bias=False)
  (conv1d): Conv1d(64, 64, kernel_size=(4,), stride=(1,), padding=(3,), groups=64)
  (act): SiLU()
  (x_proj): Linear(in_features=64, out_featu

In [10]:
%cd content

/content


In [11]:
!ls

mamba  sample_data  selective_scan_interface.py


In [12]:
!cp /content/selective_scan_interface.py /content/mamba/mamba_ssm/ops/selective_scan_interface.py

In [13]:
!cat ./mamba/mamba_ssm/ops/selective_scan_interface.py

# Copyright (c) 2023, Tri Dao, Albert Gu.

import torch
import torch.nn.functional as F
from mamba_ssm.utils.torch import custom_bwd, custom_fwd

from einops import rearrange, repeat

try:
    from causal_conv1d import causal_conv1d_fn
    from causal_conv1d.cpp_functions import causal_conv1d_fwd_function, causal_conv1d_bwd_function, \
        causal_conv1d_update_function
except ImportError:
    causal_conv1d_fn = None
    causal_conv1d_fwd_function = None
    causal_conv1d_bwd_function = None
    causal_conv1d_update_function = None

from mamba_ssm.ops.triton.layer_norm import _layer_norm_fwd

import selective_scan_cuda


class SelectiveScanFn(torch.autograd.Function):

    @staticmethod
    def forward(ctx, u, delta, A, B, C, D=None, z=None, delta_bias=None, delta_softplus=False,
                return_last_state=False):
        if u.stride(-1) != 1:
            u = u.contiguous()
        if delta.stride(-1) != 1:
            delta = delta.contiguous()
        if D is not None:
    

In [14]:
import sys
sys.path.insert(0, '/content/mamba')
from mamba_ssm import Mamba

In [15]:
import torch
model = Mamba(d_model=16, d_state=16, d_conv=4, expand=2).to("cuda")
x = torch.randn(1, 121, 16).to("cuda")
y = model(x)
print(y.shape)  # Mong đợi: torch.Size([1, 121, 16])

torch.Size([1, 121, 16])
